In [8]:
import sys
sys.path.append("/kaggle/input/marigold_model/transformers/default/2/marigold_source")
from Marigold.marigold.marigold_depth_pipeline import MarigoldDepthPipeline

In [17]:
import os
import cv2
import gc
import torch
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

#模型加载
model_path = "/kaggle/input/marigold_model/transformers/default/2/marigold_model"
pipe = MarigoldDepthPipeline.from_pretrained(model_path).to("cuda")

# ========= 路径设置 =========
base_folder = '/kaggle/input/diode-val/val/outdoor/scene_00024'
save_root = '/kaggle/working'
os.makedirs(save_root, exist_ok=True)

# ========= 可视化函数 =========
def save_comparison_figure(rgb, gt_depth, pred_depth_aligned, mask, save_path):
    plt.figure(figsize=(15, 5))
    valid = mask & (gt_depth > 0) & np.isfinite(gt_depth) & np.isfinite(pred_depth_aligned)
    if np.sum(valid) == 0:
        vmin, vmax = 0, 1  # fallback, no valid values
    else:
        all_depths = np.concatenate([gt_depth[valid], pred_depth_aligned[valid]])
        vmin = np.percentile(all_depths, 2)
        vmax = np.percentile(all_depths, 98)

    plt.subplot(1, 3, 1)
    plt.imshow(rgb)
    plt.title("RGB")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt_depth, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("GT Depth")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(pred_depth_aligned, cmap='viridis', vmin=vmin, vmax=vmax)
    plt.title("Predicted (Aligned)")
    plt.colorbar(fraction=0.046, pad=0.04)
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

# ========= 评估函数 =========
def compute_abs_rel(pred, gt, mask=None):
    # 支持 mask=None
    if mask is None:
        valid = (gt > 0)
    else:
        valid = mask & (gt > 0)
    pred, gt = pred[valid], gt[valid]
    if len(gt) == 0:
        return np.nan
    return np.mean(np.abs(pred - gt) / gt)

def compute_delta1(pred, gt, mask):
    valid_mask = mask & (gt > 1e-6) & (pred > 1e-6)
    pred, gt = pred[valid_mask], gt[valid_mask]
    thresh = np.maximum(pred / gt, gt / pred)
    return np.mean(thresh < 1.25) if len(thresh) > 0 else np.nan

def scale_match(pred, gt, mask):
    pred, gt = pred[mask], gt[mask]
    denominator = np.sum(pred ** 2)
    return np.sum(gt * pred) / denominator if denominator > 1e-8 else 1.0

# ========= 主循环 =========
scan_dirs = sorted(d for d in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, d)))

for scan_dir in scan_dirs:
    scan_path = os.path.join(base_folder, scan_dir)
    print(f"\n正在处理: {scan_dir}")
    
    vis_dir = os.path.join(save_root, scan_dir, 'visualizations')
    os.makedirs(vis_dir, exist_ok=True)
    result_rows = []

    for img_file in sorted(os.listdir(scan_path)):
        if not img_file.endswith('.png'):
            continue

        try:
            rgb_path = os.path.join(scan_path, img_file)
            gt_path = rgb_path.replace('.png', '_depth.npy')
            mask_path = rgb_path.replace('.png', '_depth_mask.npy')

            if not (os.path.exists(gt_path) and os.path.exists(mask_path)):
                print(f"跳过无 GT/mask 的图像: {img_file}")
                continue

            rgb = cv2.imread(rgb_path)
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
            gt_depth = np.load(gt_path).astype(np.float32)
            if gt_depth.ndim == 3:
                gt_depth = gt_depth[:, :, 0]
            mask = np.load(mask_path).astype(bool)

            # 推理
            input_image = Image.fromarray(rgb)
            with torch.no_grad():
                result = pipe(
                    input_image,
                    denoising_steps=4,
                    ensemble_size=1,
                    processing_res=768,
                    match_input_res=True,
                    batch_size=0,
                    show_progress_bar=False,
                )
            pred_depth = result.depth_np

            if pred_depth.shape != gt_depth.shape:
                pred_depth = cv2.resize(pred_depth, (gt_depth.shape[1], gt_depth.shape[0]), interpolation=cv2.INTER_LINEAR)

            scale = scale_match(pred_depth, gt_depth, mask)
            pred_aligned = pred_depth * scale

            abs_rel = compute_abs_rel(pred_aligned, gt_depth, mask)
            delta1 = compute_delta1(pred_aligned, gt_depth, mask)

            print(f"[{img_file}] AbsRel: {abs_rel:.4f}, δ1: {delta1*100:.2f}%")
            result_rows.append([img_file, abs_rel, delta1])
            #all_results.append([scan_dir + '/' + img_file, abs_rel, delta1])

            save_name = os.path.join(vis_dir, img_file.replace('.png', '_compare.png'))
            save_comparison_figure(rgb, gt_depth, pred_aligned, mask, save_name)

            del rgb, gt_depth, mask, pred_depth, pred_aligned, input_image

        except Exception as e:
            print(f"处理失败: {img_file}, 错误: {e}")
            continue

    # 保存当前 scan 的每图像指标
    df = pd.DataFrame(result_rows, columns=['image', 'AbsRel', 'Delta1'])
    df.to_csv(os.path.join(save_root, f"{scan_dir}_results.csv"), index=False)

    torch.cuda.empty_cache()
    gc.collect()

# 打包所有可视化图
zip_path = shutil.make_archive(os.path.join(save_root, "visualizations_all"), 'zip', root_dir=save_root)
print(f"可视化图像已打包: {zip_path}")


The config attributes {'prediction_type': 'depth'} were passed to MarigoldDepthPipeline, but are not expected and will be ignored. Please verify your model_index.json configuration file.
Keyword arguments {'prediction_type': 'depth'} are not expected by MarigoldDepthPipeline and will be ignored.


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]


正在处理: scan_00201
[00024_00201_outdoor_000_000.png] AbsRel: 0.8593, δ1: 16.70%
[00024_00201_outdoor_000_020.png] AbsRel: 1.3305, δ1: 25.05%
[00024_00201_outdoor_010_010.png] AbsRel: 1.4128, δ1: 18.28%
[00024_00201_outdoor_030_000.png] AbsRel: 0.7988, δ1: 14.87%
[00024_00201_outdoor_050_030.png] AbsRel: 1.4665, δ1: 13.02%
[00024_00201_outdoor_060_000.png] AbsRel: 0.6724, δ1: 25.69%
[00024_00201_outdoor_060_040.png] AbsRel: 1.0540, δ1: 17.77%
[00024_00201_outdoor_070_010.png] AbsRel: 0.7941, δ1: 21.72%
[00024_00201_outdoor_070_030.png] AbsRel: 1.0652, δ1: 32.69%
[00024_00201_outdoor_080_000.png] AbsRel: 0.5230, δ1: 33.05%
[00024_00201_outdoor_080_020.png] AbsRel: 0.4590, δ1: 35.02%
[00024_00201_outdoor_080_040.png] AbsRel: 0.6397, δ1: 25.55%
[00024_00201_outdoor_090_010.png] AbsRel: 0.4933, δ1: 52.42%
[00024_00201_outdoor_090_030.png] AbsRel: 0.5919, δ1: 12.11%
[00024_00201_outdoor_090_050.png] AbsRel: 0.4920, δ1: 17.62%
[00024_00201_outdoor_100_000.png] AbsRel: 0.4899, δ1: 41.61%
[00024

In [18]:

import os
import pandas as pd

output_dir = '/kaggle/working'
csv_files = []

# 搜索所有 *_results.csv 文件（包括子目录）
for root, dirs, files in os.walk(output_dir):
    for file in files:
        if file.endswith('_results.csv'):
            csv_files.append(os.path.join(root, file))

all_dfs = []
summary = []

for f in csv_files:
    df = pd.read_csv(f)
    all_dfs.append(df)
    summary.append({
        'FileName': os.path.basename(f),    # 当前csv文件名
        'NumSamples': len(df),              # 样本数量
        'MeanAbsRel': df['AbsRel'].mean(),  # AbsRel 均值
        'MeanDelta1': df['Delta1'].mean()   # Delta1 均值
    })

if all_dfs:
    # 汇总所有csv为一个总表
    df_all = pd.concat(all_dfs, ignore_index=True)
    # 汇总所有样本的均值
    total = {
        'FileName': 'ALL',
        'NumSamples': len(df_all),
        'MeanAbsRel': df_all['AbsRel'].mean(),
        'MeanDelta1': df_all['Delta1'].mean()
    }
    summary.append(total)

    # 保存到mean.csv并输出
    mean_df = pd.DataFrame(summary)
    mean_csv_path = os.path.join(output_dir, "mean.csv")
    mean_df.to_csv(mean_csv_path, index=False)
    print(mean_df)
    print(f"所有scan子文件统计均值已保存到: {mean_csv_path}")
else:
    print("未找到任何*_results.csv结果文件！")


                  FileName  NumSamples  MeanAbsRel  MeanDelta1
0   scan_00201_results.csv          43    0.786545    0.273243
1   scan_00199_results.csv          38    0.454627    0.372933
2   scan_00194_results.csv          44    1.011522    0.253333
3   scan_00189_results.csv          22    0.408705    0.439478
4   scan_00195_results.csv          52    0.483996    0.364701
5   scan_00196_results.csv          53    0.689669    0.397270
6   scan_00183_results.csv          73    0.312922    0.531335
7   scan_00192_results.csv          25    0.377369    0.415917
8   scan_00191_results.csv          18    0.389166    0.390327
9   scan_00190_results.csv          25    0.366060    0.460875
10  scan_00185_results.csv          29    0.361463    0.449896
11  scan_00187_results.csv          33    0.345724    0.456646
12  scan_00193_results.csv          51    0.753206    0.300258
13  scan_00200_results.csv          40    0.644125    0.351994
14  scan_00184_results.csv          27    0.309433    0